In [ ]:
%load_ext autoreload

%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
from skimage.exposure import rescale_intensity

from sentinelhub import CRS, BBox, DataCollection, SHConfig
from sentinelhub import SentinelHubCatalog
from sentinelhub import SHConfig
from oxeo.core.stac import landsat
from oxeo.core.stac.constants import USWEST_URL ,ELEMENT84_URL
uswest_config = SHConfig()
uswest_config.sh_base_url = USWEST_URL

uswest_catalog = SentinelHubCatalog(config=uswest_config)
eu_catalog = SentinelHubCatalog(SHConfig())
from oxeo.core.data import get_aoi_from_landsat_shub_catalog, get_aoi_from_stac_catalog
import os
os.environ["AWS_REQUEST_PAYER"] = "requester"

from oxeo.core.utils import get_bounding_box

In [ ]:
box = get_bounding_box({"geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -107.96882629394531,
              29.209713225868185
            ],
            [
              -107.70927429199219,
              29.209713225868185
            ],
            [
              -107.70927429199219,
              29.482643134466617
            ],
            [
              -107.96882629394531,
              29.482643134466617
            ],
            [
              -107.96882629394531,
              29.209713225868185
            ]
          ]
        ]
      }})

In [ ]:
bbox = BBox(box, crs=CRS.WGS84)#BBox([49.9604, 44.7176, 51.0481, 45.2324], crs=CRS.WGS84) #BBox((-71.40254974365233, -46.9537775782648, -71.26213073730467, -46.89163931213445), crs=CRS.WGS84)#
time_interval = "2020-12-10", "2021-02-01" #"1994-12-10", "1995-02-01"
search_params = {
}


In [ ]:
s1_aoi = get_aoi_from_stac_catalog(catalog=eu_catalog,
                                   data_collection=DataCollection.SENTINEL1,
                                   bbox=bbox,
                                   time_interval=time_interval,
                                   search_params={})


                                        


In [ ]:
img = s1_aoi.sel(band="vv").mean(axis=0)


In [ ]:
db = 10*np.log10(img)
plt.imshow(db)

In [ ]:
s2_aoi = get_aoi_from_stac_catalog(catalog=ELEMENT84_URL,
                                   data_collection="sentinel-s2-l2a-cogs",
                                   bbox=bbox,
                                   time_interval=time_interval,
                                   search_params={})

img = s2_aoi.sel(band=["B04","B03","B02"])[:5].mean(axis=0)[:,:,:].values.transpose(1,2,0)
plt.imshow(rescale_intensity(img,(np.nanmin(img), np.nanmax(img))))

In [ ]:
landsat_aoi = get_aoi_from_stac_catalog(catalog=uswest_catalog,
                                        data_collection=DataCollection.LANDSAT_OT_L2,
                                        bbox=bbox,
                                        time_interval=time_interval,
                                        search_params={},
                                        category="T1")
                                        

img = landsat_aoi.sel(band=['SR_B4', 'SR_B3', 'SR_B2'])[:5].mean(axis=0)[:,:,:].values.transpose(1,2,0)
plt.imshow(rescale_intensity(img,(np.nanmin(img), np.nanmax(img))))
